# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,166.130810,-0.363744,1234,165.767065,166.494554,331.534131,332.989107
6,94.166209,-0.250849,1234,93.915360,94.417058,187.830719,188.834116
7,224.158138,0.568383,1234,224.726521,223.589755,449.453042,447.179511
8,382.455625,0.327679,1234,382.783305,382.127946,765.566609,764.255892
9,778.842263,-0.103109,1234,778.739153,778.945372,1557.478307,1557.890744
10,288.397689,0.395972,1234,288.793661,288.001717,577.587323,576.003434
11,398.081454,-0.402145,1234,397.679309,398.483598,795.358618,796.967197
12,854.900555,-0.803006,1234,854.097549,855.703561,1708.195098,1711.407122
13,54.422947,0.508226,1234,54.931173,53.914721,109.862346,107.829442
14,51.114335,-0.860859,1234,50.253476,51.975194,100.506951,103.950389


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-165.767065,166.494554
6,-93.915360,94.417058
7,-224.726521,223.589755
8,-382.783305,382.127946
9,-778.739153,778.945372
10,-288.793661,288.001717
11,-397.679309,398.483598
12,-854.097549,855.703561
13,-54.931173,53.914721
14,-50.253476,51.975194


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-165.767065,166.494554
6,-93.915360,94.417058
7,-224.726521,223.589755
8,-382.783305,382.127946
9,-778.739153,778.945372
10,-288.793661,288.001717
11,-397.679309,398.483598
12,-854.097549,855.703561
13,-54.931173,53.914721
14,-50.253476,51.975194


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-165.76706547,  166.49455373],
       [ -93.91535954,   94.41705809],
       [-224.72652076,  223.58975538],
       [-382.78330463,  382.12794589],
       [-778.73915347,  778.94537182],
       [-288.79366131,  288.00171689],
       [-397.67930879,  398.48359835],
       [-854.09754911,  855.70356097],
       [ -54.9311731 ,   53.91472104],
       [ -50.25347555,   51.97519433]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,329.534131,-332.989107
6,185.830719,-188.834116
7,447.453042,-447.179511
8,763.566609,-764.255892
9,1555.478307,-1557.890744
10,575.587323,-576.003434
11,793.358618,-796.967197
12,1706.195098,-1711.407122
13,107.862346,-107.829442
14,98.506951,-103.950389


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
